In [33]:
import pandas as pd

- `results.csv` joins `races.csv` to get `year`
- `results.csv` joins `drivers.csv` to get `driver`
- `results.csv` joins `constructors.csv` to get `constructor`
- `results.csv` joins `pitStops.csv` to get `duration`

In [34]:
results = pd.read_csv('data/results.csv')
results = results[['raceId', 'driverId', 'constructorId', 'position', 'rank']]

races = pd.read_csv('data/races.csv')
races.rename(columns={'name': 'raceName'}, inplace=True)
races = races[['raceId', 'year', 'raceName']]

drivers = pd.read_csv('data/drivers.csv')
drivers['driverFullname'] = drivers['forename'] + ' ' + drivers['surname']
drivers = drivers[['driverId', 'driverFullname']]
    
constructors = pd.read_csv('data/constructors.csv')
constructors.rename(columns={'name': 'constructorName'}, inplace=True)
constructors = constructors[['constructorId', 'constructorName']]

pitstops = pd.read_csv('data/pit_stops.csv')
pitstops = pitstops[['raceId', 'driverId', 'stop', 'duration', 'milliseconds']]

In [35]:
results = results.merge(races, on='raceId', how='inner')
results = results.merge(drivers, on='driverId', how='left')
results = results.merge(constructors, on='constructorId', how='left')
results = results.merge(pitstops, on=['raceId', 'driverId'], how='left')

---

### Getting `times.csv`

In [36]:
columnsTimes = ['year', 'raceName', 'driverFullname', 'constructorName', 'position', 'stop', 'milliseconds']
times = results[columnsTimes].sort_values(by=['year']) \
                             .dropna(subset=['milliseconds'])

times['duration'] = times['milliseconds'] / 1000

times.to_csv('data/times.csv', index=False)
times.to_csv('../Public-Viz-F1/times.csv', index=False)

---
### Getting `driversWinners.csv` & `constructorsWinners.csv`

In [37]:
columnsDriverWinners = ['year', 'raceId', 'raceName', 'driverFullname', 'constructorName', 'position']

driversWinners = results[columnsDriverWinners].query('position == "1"') \
                                              .drop_duplicates() \
                                              .groupby(['year', 'driverFullname'])['position'] \
                                              .count() \
                                              .reset_index(name='wins')
                                              
# make the number of wins be cumulative
driversWinners['wins'] = driversWinners.groupby(['driverFullname'])['wins'].cumsum()
                                              
driversWinners.to_csv('data/driversWinners.csv', index=False)
driversWinners.to_csv('../Public-Viz-F1/driversWinners.csv', index=False)

driversWinners

,year,driverFullname,wins
0,1950,Johnnie Parsons,1
1,1950,Juan Fangio,3
2,1950,Nino Farina,3
3,1951,Alberto Ascari,2
4,1951,José Froilán González,1
...,...,...,...
388,2022,Carlos Sainz,1
389,2022,Charles Leclerc,5
390,2022,George Russell,1
391,2022,Max Verstappen,35


In [38]:
columnsConstructorsWinners = ['year', 'raceId', 'raceName', 'constructorName', 'position']

constructorsWinners = results[columnsConstructorsWinners].query('position == "1"') \
                                                         .drop_duplicates() \
                                                         .groupby(['year', 'constructorName'])['position'] \
                                                         .count() \
                                                         .reset_index(name='wins')
                                                         
# make the number of wins be cumulative
constructorsWinners['wins'] = constructorsWinners.groupby(['constructorName'])['wins'].cumsum()

constructorsWinners.to_csv('data/constructorsWinners.csv', index=False)
constructorsWinners.to_csv('../Public-Viz-F1/constructorsWinners.csv', index=False)

constructorsWinners

,year,constructorName,wins
0,1950,Alfa Romeo,6
1,1950,Kurtis Kraft,1
2,1951,Alfa Romeo,10
3,1951,Ferrari,3
4,1951,Kurtis Kraft,2
...,...,...,...
266,2021,Mercedes,124
267,2021,Red Bull,75
268,2022,Ferrari,242
269,2022,Mercedes,125


In [39]:
# make the number of wins be cumulative and be counted until the current year
constructorsWinners['wins'] = constructorsWinners.groupby(['constructorName'])['wins'].cumsum()